In [1]:
%load_ext autoreload
%autoreload 2

## Learning a classifier

On commence par la fonction de base apprise de la librairie torch, cf https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py :

In [2]:
%run what.py --epochs 1 --save-model

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213460
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170403
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076578
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.867874
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.413523
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000300
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.776155
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.459784
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.485992
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438883
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.408428
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.460157
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.428865
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.398983
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.384978
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.298446
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.503128
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.523736
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.338001
Train Epoch: 1 [12800/60000 (


On apprend une matrice de poids qui est fixée dans la suite et que nous allons utiliser pour la classification

In [9]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
default = dict(batch_size=64, test_batch_size=1000, epochs=1, no_cuda=True, seed=42, lr=0.01, momentum=.5, save_model=True)

In [26]:
parser.parse_args?

Signature: parser.parse_args(args=None, namespace=None)
Docstring: <no docstring>
File:      /usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/argparse.py
Type:      method


In [30]:
args = parser.parse_args(args=default)

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: batch_size test_batch_size epochs no_cuda seed lr momentum save_model


SystemExit: 2

In [29]:
args.lr

AttributeError: 'tuple' object has no attribute 'lr'

In [31]:
from what import parse

In [32]:
parser = parse()

In [34]:
args = parser.parse_args(namespace=default)

AttributeError: 'dict' object has no attribute 'batch_size'

In [3]:
from what import run

In [5]:
args = parse()

usage: ipykernel_launcher.py [-h] [--batch-size N] [--test-batch-size N]
                             [--epochs N] [--lr LR] [--momentum M] [--no-cuda]
                             [--seed S] [--log-interval N] [--save-model]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/laurentperrinet/Library/Jupyter/runtime/kernel-696ed243-e233-4fc1-b07c-c71154e6e4cb.json


SystemExit: 2

In [ ]:
%ls -ltr ../data/mnist_cnn.pt


In [ ]:
path = "../data/mnist_cnn.pt"

import os
if os.path.isfile(path):
    print('Loading')
    model.load_state_dict(torch.load(path))
else:
    print('Learning')
    import time
    t0 = time.time()
    for epoch in range(1, args.epochs + 1):
        train(epoch)
    torch.save(model.state_dict(), path)
    
    print('Done in ', time.time() - t0, 'seconds')

In [ ]:
test()

## Using the learned classifier in a standalone class

Maintenant qu'on a appris les points qui permet une classification d'à peu près 98 % on va utiliser le modèle fead-forward pour faire la classification.

In [ ]:
import torch.nn as nn

class ParvoNet(nn.Module):
    def __init__(self):
        super(ParvoNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

import torch
test_batch_size = 1000
test_batch_size = 20
cmin, cmax = 0.1307, 0.3081
test_loader = torch.utils.data.DataLoader(
datasets.MNIST('/tmp/data', train=False, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((cmin,), (cmax,))
               ])), batch_size=test_batch_size, shuffle=True, **kwargs)


def test(test_loader=test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


model = ParvoNet()
path = "MNIST_classifier.pt"
model.load_state_dict(torch.load(path))

test()


## shifting the input images


Je vais maintenant générer des données en utilisant les données originales de MNIST translatées :

In [ ]:

from torchvision import datasets, transforms
i_shift, j_shift = 12, 17
N_pix = 28


In [ ]:
%matplotlib inline
for data, target in test_loader:
    break
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.imshow(data[0, 0, :, :])

In [ ]:
import numpy as np
data_translate = data.min() * np.ones((data.shape[0], 1, N_pix*3 - 2, N_pix*3 - 2))
print(data_translate.shape)
data_translate[:, :, (N_pix-i_shift):(2*N_pix-i_shift), (N_pix-j_shift):(2*N_pix-j_shift)] = data
fig, ax = plt.subplots()
ax.imshow(data_translate[0, 0, :, :])

In [ ]:
data_cropped = data_translate[:, :, (N_pix):(2*N_pix), (N_pix):(2*N_pix)]
fig, ax = plt.subplots()
ax.imshow(data_cropped[0, 0, :, :])

In [ ]:
np.arange(-N_pix+1, N_pix)

In [ ]:
def shift_data(data, i_shift, j_shift):
    N_pix = data.shape[-1]
    assert(N_pix == data.shape[-2])
    import numpy as np
    data_translate = data.min() * np.ones((data.shape[0], 1, N_pix*3 - 1, N_pix*3 - 1))
    data_translate[:, :, (N_pix+i_shift):(2*N_pix+i_shift), (N_pix+j_shift):(2*N_pix+j_shift)] = data
    data_cropped = data_translate[:, :, (N_pix):(2*N_pix), (N_pix):(2*N_pix)]
    return data_cropped

data_cropped = shift_data(data, i_shift = 12, j_shift = -12)
fig, ax = plt.subplots()
ax.imshow(data_cropped[0, 0, :, :])

## Using the learned classifier on the shifted data

On peut maintenant tester le classifieur sur les images Translatées en calculant la valeur de classification en  fonction de l'erreur de localisation

In [ ]:
def test_shift(test_loader, i_shift, j_shift, verbose=0):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data_cropped = shift_data(data, i_shift=i_shift, j_shift=j_shift)        
        data_cropped = torch.FloatTensor(data_cropped) #transforms.ToTensor()(data_cropped)
        data_cropped, target = Variable(data_cropped, volatile=True), Variable(target)
        output = model(data_cropped)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    if verbose: print('\nTest set: at ({}, {}), the  average loss is {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        i_shift, j_shift, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


path = "MNIST_accuracy.npy"

import os
if os.path.isfile(path):
    print('Loading accuracy')
    accuracy = np.load(path)
else:
    print('Computing accuracy')
    import time
    t0 = time.time()
    accuracy = np.zeros((2*N_pix-1, 2*N_pix-1))
    from tqdm import tqdm
    N_step = 1

    with tqdm(total=(2*N_pix-1)**2/N_step**2) as pbar:
        for i_shift in np.arange(-N_pix+1, N_pix, N_step):
            for j_shift in np.arange(-N_pix+1, N_pix, N_step):
                accuracy[i_shift+N_pix-1, j_shift+N_pix-1] = test_shift(test_loader, i_shift, j_shift)
                pbar.update()
    np.save(path, accuracy)
    print('Done in ', time.time() - t0, 'seconds')
    

J'avoue que c'est un peu bourrin de calculer la classification sur les 128 × 128 pixels pour 1000 batch multiplié par 10 type d'entrées.... Mais bon on doit faire ça seulement une fois :-) (et sur CPU une classif = environ 300µs ...)

In [ ]:
help(plt.pcolor)


In [ ]:
fig, ax = plt.subplots(figsize=(13, 10.725))
cmap = ax.pcolor(np.arange(-N_pix+1, N_pix+1), np.arange(-N_pix+1, N_pix+1), accuracy)
ax.axis('equal')
fig.colorbar(cmap)

In [ ]:
accuracy

Cette fonction de performance du classifieur  est calculée indépendamment de la forme spécifique du chiffre entre 0 et 9. Elle donne donc la carte de performance qu'on attend Au niveau de la classification/ On va pouvoir maintenant l'utiliser ceomm label pour apprendre de façon supervisée la correspondance entre la carte log-polaire obtenue depuis l'image brute et cette carte de performance.


# Retinotopic mapping

#### Orientation invariant power encoding (colliculus??)

In [ ]:
colliculus = (retina_transform**2).sum(axis=(0, 3))
#colliculus = colliculus**.5
colliculus /= colliculus.sum(axis=-1)[:, :, None]
print(colliculus.shape)

In [ ]:
colliculus_vector = colliculus.reshape((N_azimuth*N_eccentricity, N_X*N_Y))
print(colliculus_vector.shape)

In [ ]:
colliculus_inverse = np.linalg.pinv(colliculus_vector)
print(colliculus_inverse.shape)

In [ ]:
energy = (retina**2).sum(axis=(0,3)) 
energy /= energy.sum(axis=-1)[:, :, None]
energy_vector = energy.reshape((N_azimuth*N_eccentricity, N_X*N_Y))
energy_plus = np.linalg.pinv(energy_vector)
FIG_WIDTH = 5 

fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_WIDTH))
for i_orient in range(N_azimuth):
    for i_scale in range(N_eccentricity):
        env = np.sqrt(energy[i_orient, i_scale, :]**2.5).reshape((N_X, N_Y))
        ax.contour(energy[i_orient, i_scale, :].reshape((N_X, N_Y)), levels=[env.max()/2], lw=1,
                  colors=[plt.cm.rainbow(i_scale * 1.5/N_azimuth)])
fig.suptitle('Tiling of visual space using energy', y=1.02)
ax.set_xlabel(r'$Y$')
ax.set_ylabel(r'$X$')
ax.axis('equal')
plt.tight_layout()